In [1]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/My Drive/Kaggle"

In [2]:
%cd /content/drive/My Drive/Kaggle

/content/drive/My Drive/Kaggle


In [3]:
import os
import matplotlib.pyplot as plt
import cv2
import numpy as np
import matplotlib.gridspec as gridspec
import seaborn as sns
import zlib
import itertools
import sklearn
import itertools
import scipy
import skimage
from skimage.transform import resize
import csv
from tqdm import tqdm
from sklearn import model_selection
from sklearn.model_selection import train_test_split, learning_curve,KFold,cross_val_score,StratifiedKFold
from sklearn.utils import class_weight
from sklearn.metrics import confusion_matrix
import keras
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, Lambda, MaxPool2D, BatchNormalization
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras import models, layers, optimizers
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.utils import class_weight
from keras.optimizers import SGD, RMSprop, Adam, Adagrad, Adadelta, RMSprop
from keras.models import Sequential, model_from_json
from keras.layers import Activation,Dense, Dropout, Flatten, Conv2D, MaxPool2D,MaxPooling2D,AveragePooling2D, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint,Callback, EarlyStopping, ReduceLROnPlateau
from keras import backend as K
import tensorflow as tf
import pickle as pkl
from keras.layers import Dense
import pickle
from tqdm import tqdm
import random
random.seed(123)

In [ ]:
with open("/content/drive/My Drive/Kaggle/train.pkl", "rb") as f1:
    x_train, y_train = pkl.load(f1)

In [ ]:

with open("/content/drive/My Drive/Kaggle/test.pkl", "rb") as f2:
    x_test, y_test = pkl.load(f2)


In [ ]:
with open("/content/drive/My Drive/Kaggle/val.pkl", "rb") as f3:
    x_val, y_val = pkl.load(f3)

In [ ]:
batch_size=32
train_datagen = ImageDataGenerator(
        rescale=1.0/255,  
        height_shift_range= 0.05, 
        width_shift_range=0.1, 
        rotation_range=15,
        shear_range=0.2,
        zoom_range=0.1,
        horizontal_flip=True,fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale=1.0/255)

In [ ]:
train_batch = train_datagen.flow(x_train,y_train,batch_size=batch_size,shuffle=True)
test_batch = test_datagen.flow(x_test,y_test,batch_size=batch_size,shuffle=False)
val_batch = train_datagen.flow(x_val,y_val,batch_size=batch_size,shuffle=False)

In [ ]:
model6=Sequential()
model6.add(Conv2D(32,(3,3),activation='relu', input_shape=(128, 128, 3),padding='same'))
model6.add(BatchNormalization())
model6.add(MaxPooling2D((2, 2)))


model6.add(Conv2D(64,(3,3),activation='relu',padding='same'))
model6.add(BatchNormalization())
model6.add(MaxPooling2D((2, 2)))

model6.add(Conv2D(128,(3,3),activation='relu',padding='same'))
model6.add(BatchNormalization())
model6.add(MaxPooling2D((2, 2)))

model6.add(Conv2D(128,(3,3),activation='relu',padding='same'))
model6.add(BatchNormalization())
model6.add(MaxPooling2D((2, 2)))

model6.add(Conv2D(256,(3,3),activation='relu',padding='same'))
model6.add(BatchNormalization())
model6.add(MaxPooling2D((2, 2)))

model6.add(Conv2D(256,(3,3),activation='relu',padding='same'))
model6.add(BatchNormalization())
model6.add(MaxPooling2D((2, 2)))

model6.add(Conv2D(512,(3,3),activation='relu',padding='same'))
model6.add(BatchNormalization())
model6.add(MaxPooling2D((2, 2)))

model6.add(Flatten())
model6.add(Dense(512, activation='relu'))
model6.add(BatchNormalization())
model6.add(Dense(256, activation='relu'))
model6.add(BatchNormalization())
model6.add(Dropout(0.2))
model6.add(Dense(4,activation='softmax'))
model6.summary()

In [ ]:
model_img_file = '/content/drive/My Drive/Colab Notebooks/model-7c12-1.png'
tf.keras.utils.plot_model(model6, to_file=model_img_file, show_shapes=True, show_layer_names=True)
plt.savefig(model_img_file )

In [ ]:
model6.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(lr=0.0001),metrics=['accuracy'])

In [ ]:
file_path="/content/drive/My Drive/Colab Notebooks/best52.hdf5"
llr=ReduceLROnPlateau(monitor='val_loss',factor=0.1,verbose=1, mode='auto',patience=5,min_delta=0.0001,min_lr=1e-5, cooldown=0)
checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
early = EarlyStopping(monitor="val_loss", mode='auto', patience=8,restore_best_weights=True)
callbacks_list = [llr,checkpoint,early]

In [ ]:
history=model6.fit_generator(train_batch, validation_data=val_batch, epochs=100, validation_steps= x_val.shape[0] // batch_size, 
                       steps_per_epoch= x_train.shape[0] // batch_size, callbacks=[callbacks_list], verbose=1)

In [ ]:
def plot_learning_curve(history):
    plt.figure(figsize=(18,9))
    plt.subplot(1,2,1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.savefig('/content/drive/My Drive/Colab Notebooks/accuracy_curve22.png')
    plt.subplot(1,2,2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.savefig('/content/drive/My Drive/Colab Notebooks/loss_curve22.png')

In [ ]:
plot_learning_curve(history)
plt.show()

In [ ]:
model6.load_weights("/content/drive/My Drive/Colab Notebooks/best52.hdf5")
score=model6.evaluate(test_batch)
print(score[1])

In [ ]:
y_pred = model6.predict_generator(test_batch,steps = 968)
Y_pred = np.argmax(y_pred,axis = 1)
Y_true = np.argmax(y_test,axis = 1)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(Y_true,Y_pred,target_names=['CNV','DME','DRUSEN','Normal']))

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.savefig('/content/drive/My Drive/Colab Notebooks/confusion22.png')

In [ ]:
# confusion matrix
cm = confusion_matrix(Y_true, Y_pred)
 
print('--------------------')
print('| Confusion Matrix |')
print('--------------------')
print('\n {}'.format(cm))
        
# plot confusin matrix
plt.figure(figsize=(8,8))
plt.grid(b=False)
plot_confusion_matrix(cm, classes=['CNV','DME','DRUSEN','NORMAL'], normalize=False, 
                      title='Confusion matrix', cmap = plt.cm.Blues)
plt.show()

In [ ]:
model6.save('/content/drive/My Drive/Colab Notebooks/7c12-0001.h5')

In [ ]:
model7=Sequential()
model7.add(Conv2D(32,(3,3),activation='relu', input_shape=(128, 128, 3),padding='same'))
model7.add(BatchNormalization())
model7.add(MaxPooling2D((2, 2)))

model7.add(Conv2D(64,(3,3),activation='relu',padding='same'))
model7.add(BatchNormalization())
model7.add(MaxPooling2D((2, 2)))

model7.add(Conv2D(128,(3,3),activation='relu',padding='same'))
model7.add(BatchNormalization())
model7.add(MaxPooling2D((2, 2)))

model7.add(Conv2D(128,(3,3),activation='relu',padding='same'))
model7.add(BatchNormalization())
model7.add(MaxPooling2D((2, 2)))

model7.add(Conv2D(256,(3,3),activation='relu',padding='same'))
model7.add(BatchNormalization())
model7.add(MaxPooling2D((2, 2)))

model7.add(Conv2D(256,(3,3),activation='relu',padding='same'))
model7.add(BatchNormalization())
model7.add(MaxPooling2D((2, 2)))

model7.add(Conv2D(512,(3,3),activation='relu',padding='same'))
model7.add(BatchNormalization())
model7.add(MaxPooling2D((2, 2)))

model7.add(Flatten())
model7.add(Dense(512, activation='relu'))
model7.add(BatchNormalization())
model7.add(Dense(256, activation='relu'))
model7.add(BatchNormalization())
model7.add(Dropout(0.4))
model7.add(Dense(4,activation='softmax'))
model7.summary()

In [ ]:
model_img_file = '/content/drive/My Drive/Colab Notebooks/model-7c12-2.png'
tf.keras.utils.plot_model(model7, to_file=model_img_file, show_shapes=True, show_layer_names=True)
plt.savefig(model_img_file )

In [ ]:
model7.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(lr=0.0001),metrics=['accuracy'])

In [ ]:
file_path="/content/drive/My Drive/Colab Notebooks/best62.hdf5"
llr=ReduceLROnPlateau(monitor='val_loss',factor=0.1,verbose=1, mode='auto',patience=5,min_delta=0.0001,min_lr=1e-6, cooldown=0)
checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
early = EarlyStopping(monitor="val_loss", mode='auto', patience=10,restore_best_weights=True)
callbacks_list = [llr,checkpoint,early]

In [ ]:
history=model7.fit_generator(train_batch, validation_data=val_batch, epochs=100, validation_steps= x_val.shape[0] // batch_size, 
                       steps_per_epoch= x_train.shape[0] // batch_size, callbacks=[callbacks_list], verbose=1)

In [ ]:
def plot_learning_curve(history):
    plt.figure(figsize=(18,9))
    plt.subplot(1,2,1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.savefig('/content/drive/My Drive/Colab Notebooks/accuracy_curve32.png')
    plt.subplot(1,2,2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.savefig('/content/drive/My Drive/Colab Notebooks/loss_curve32.png')

In [ ]:
plot_learning_curve(history)
plt.show()

In [ ]:
model7.load_weights("/content/drive/My Drive/Colab Notebooks/best62.hdf5")
score=model7.evaluate(test_batch)
print(score[1])

In [ ]:
y_pred = model7.predict_generator(test_batch,steps = 968)
Y_pred = np.argmax(y_pred,axis = 1)
Y_true = np.argmax(y_test,axis = 1)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(Y_true,Y_pred,target_names=['CNV','DME','DRUSEN','Normal']))

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.savefig('/content/drive/My Drive/Colab Notebooks/confusion34.png')

In [ ]:
# confusion matrix
cm = confusion_matrix(Y_true, Y_pred)
 
print('--------------------')
print('| Confusion Matrix |')
print('--------------------')
print('\n {}'.format(cm))
        
# plot confusin matrix
plt.figure(figsize=(8,8))
plt.grid(b=False)
plot_confusion_matrix(cm, classes=['CNV','DME','DRUSEN','NORMAL'], normalize=False, 
                      title='Confusion matrix', cmap = plt.cm.Blues)
plt.show()

In [ ]:
model7.save('/content/drive/My Drive/Colab Notebooks/7c1-00012-2.h5')